<a href="https://colab.research.google.com/github/Waseem-687/Agentic-AI/blob/main/LangChain_Rag_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec
pinecone_api_key = userdata.get('PINECONE_API_KEY')
pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
import time

index_name = "langchan-rag-project"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("white, flower!")
vector[:3]

[0.05607491731643677, -0.0688314214348793, -0.06224929541349411]

In [ ]:
vector = embeddings.embed_query("who was the shakespeare?!")
vector[:3]

[0.008599186316132545, -0.05972126126289368, -0.014227797277271748]

In [ ]:
vector = embeddings.embed_query("who was the shakespeare?!")
vector[:3]

[0.008599186316132545, -0.05972126126289368, -0.014227797277271748]

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['c67d92a8-5634-48ae-96c2-cb7824d9b5ec',
 '4e8de054-1549-439d-864c-55d7a599ce9a',
 'a57237ca-3c33-4b7e-ab6a-a111f4ae2625',
 '2ac168b5-46ab-4933-bac3-ea2f747c8faa',
 '4fe9fe60-07d1-42b1-b96e-4b6f98d33dc7',
 'a68f0066-83d4-4650-ba8e-27a54e35dc46',
 '2d6d47d5-6c6c-49e5-8bd8-3309f5f1c258',
 '6121edde-1297-42fa-953a-fa43be82e4af',
 'cf045805-f7b2-496c-af52-d6d3618497b1',
 '6dcd9ae4-4609-4ec6-9197-4cb1ab9b5bca']

In [ ]:
len(documents)

10

In [ ]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['7b503336-43a2-4ef3-b8d9-d100fa231518',
 'd79102e7-95b3-4544-bc3d-e573584b8115',
 '44463858-2364-4284-bb80-eab2f8b24952',
 '0bfa697e-d032-437c-a2fc-b98fae4904a1',
 '8a7a7027-1df9-4efb-a325-71ec852dffe4',
 '88f855d6-ea51-411a-8d7a-d26981d1b998',
 '684e6e9b-2cb1-46e5-8dd2-b73be46625e2',
 '07a46690-d01f-4f97-9e93-fb42d5e4d637',
 '8d9459fe-2a9c-4330-8a7a-69f10d45d8f7',
 '3353961c-c7af-481d-a1a8-29cd26bc9074']

In [24]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=6,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


In [25]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=7, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.668031] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.668031] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.668031] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.668031] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.668031] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.534239] The stock market is down 500 points today due to fears of a recession. [{'source': 'news'}]
* [SIM=0.534239] The stock market is down 500 points today due to fears of a recession. [{'source': 'news'}]


In [26]:
def answer_to_user_question(question: str) -> str:
    results = vector_store.similarity_search(question, k=5)
    final_answer = model(results, question)
    return final_answer